Linear regression to predict streamflow. Forcings are aggregated from hourly to daily to match streamflow resolution.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn import metrics, svm, neural_network, ensemble
from datetime import datetime, timedelta
import hydroeval
import netCDF4 as nc
from src import load_data, evaluate

In [2]:
data_runoff = load_data.load_discharge_gr4j_vic()

/home/mgauch/runoff-nn/src/load_data.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(os.path.join(dir, f), skiprows=2, skipfooter=1, index_col=False, header=None, names=['runoff'], na_values='-1.2345')
/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
data_runoff[pd.isna(data_runoff['runoff'])].groupby(['station'])['date'].count()  # Count NAs per station

station
02GC018       44
02GG013       31
04214500    1003
Name: date, dtype: int64

In [4]:
# For each station, read which grid cells belong to its subwatershed
station_cell_mapping = pd.read_csv('station_cell_mapping.csv', skiprows=1, names=['station', 'lat', 'lon', 'row', 'col', 'area'])

In [5]:
rdrs_data = load_data.load_rdrs_forcings()

In [6]:
resampled = rdrs_data.resample('D')
rdrs_daily = resampled.sum().join(resampled.min(), lsuffix='_sum', rsuffix='_min').join(resampled.max().rename(lambda c: c + '_max', axis=1))

In [ ]:
#pca = PCA(10)
#pca_result = pca.fit_transform(data.drop(['date', 'runoff', 'station'], axis=1))
#sum(pca.explained_variance_ratio_)
#data = data[['date', 'runoff', 'station']].join(pd.DataFrame(pca_result))

In [7]:
# Create test and train splits for each station (by time), then create predictions for each subbasin
history = 7
train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=history + 1)
use_runoff_history = False

predictions = {}
actuals = {}
for station in data_runoff['station'].unique():
    print(station)
    station_data = data_runoff[data_runoff['station'] == station].set_index('date')
    station_cell_ids = 39 * station_cell_mapping[station_cell_mapping['station'] == station]['col'] \
        + station_cell_mapping[station_cell_mapping['station'] == station]['row']
    
    # For temperature use min/max aggregation. Precipitation: sum. solar fluxes, pressure & humidity don't seem to help (at least with min/max/sum)
    regex = '((RDRS_TT_40m)_({0})_(min|max)|(RDRS_PR0_SFC)_({0})_sum)$'.format('|'.join(map(lambda x: str(x), station_cell_ids)))
    station_rdrs = rdrs_daily.filter(regex=regex, axis=1)#'_(' + '|'.join(map(lambda x: str(x), station_cell_ids)) + ')_(mean|min|max)$', axis=1)
    print(station_rdrs.shape)
    if any(station_data['runoff'].isna()):
        station_data = station_data[~pd.isna(station_data['runoff'])]
        print('Station', station, 'had NA runoff values')
    
    station_data = station_data.join(station_rdrs)
    for i in range(1, history + 1):
        station_data = station_data.join(station_rdrs.shift(i, axis=0), rsuffix='_-{}'.format(i))
        if use_runoff_history:
            station_data['runoff_-{}'.format(i)] = station_data['runoff'].shift(i, axis=0)
    
    station_train = station_data.loc[train_start : '2013-12-31']
    station_test = station_data.loc['2014-01-01' : '2014-12-31']
    
    print('  Fitting model')
    m = linear_model.LinearRegression()
    m.fit(station_train.drop(['station', 'runoff'], axis=1), station_train['runoff'])
    
    station_test = station_test[~pd.isna(station_test['runoff'])]
    if len(station_test) == 0:
        print('Skipping', station)
        continue
        
    predict = pd.DataFrame(index=station_test.index)
    predict = predict.join(station_test.drop(['station', 'runoff'], axis=1))
    predict['runoff'] = np.nan
    if use_runoff_history:
        for i in range(history):
            for j in range(i + 1, history + 1):
                predict.iloc[i]['runoff_-{}'.format(j)] = station_test.iloc[i]['runoff_-{}'.format(j)]
    print('  Predicting')
    if not use_runoff_history:
        predict['runoff'] = m.predict(predict.drop('runoff', axis=1))
    else:
        for i in range(len(predict)):
            predict.iloc[i]['runoff'] = m.predict([predict.iloc[i].drop('runoff')])[0]
            for j in range(1, history + 1):
                if (i + j) >= len(predict):
                    break
                predict.iloc[i + j]['runoff_-{}'.format(j)] = predict.iloc[i]['runoff']
    predictions[station] = predict
    actuals[station] = station_test['runoff']

04159492
(1827, 42)
  Fitting model
  Predicting
04200500
(1827, 27)
  Fitting model
  Predicting
04161820
(1827, 27)
  Fitting model
  Predicting
04174500
(1827, 51)
  Fitting model
  Predicting
02GC010
(1827, 18)
  Fitting model
  Predicting
02GE007
(1827, 18)
  Fitting model
  Predicting
02GD004
(1827, 18)
  Fitting model
  Predicting
04214500
(1827, 27)
Station 04214500 had NA runoff values
  Fitting model
  Predicting
04213000
(1827, 27)
  Fitting model
  Predicting
02GG009
(1827, 30)
  Fitting model
  Predicting
02GA010
(1827, 36)
  Fitting model
  Predicting
04176500
(1827, 72)
  Fitting model
  Predicting
04177000
(1827, 15)
  Fitting model
  Predicting
02GA047
(1827, 30)
  Fitting model
  Predicting
02GC018
(1827, 21)
Station 02GC018 had NA runoff values
  Fitting model
  Predicting
04166500
(1827, 30)
  Fitting model
  Predicting
04207200
(1827, 15)
  Fitting model
  Predicting
04160600
(1827, 18)
  Fitting model
  Predicting
04215000
(1827, 15)
  Fitting model
  Predicting
0

In [8]:
# Evaluate each station
nse_list = []
plot_list = ['04159492', '02GG013']
for station, predict in predictions.items():
    nse = evaluate.evaluate_daily(station, predict['runoff'], actuals[station], plot=station in plot_list)
    nse_list.append(nse)
    
    print(station, '\tNSE: (clipped to 0)', nse_list[-1])

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))

04159492 	NSE: (clipped to 0) -0.4836861795641103
04200500 	NSE: (clipped to 0) 0.39863295389455033
04161820 	NSE: (clipped to 0) 0.26187359116312103
04174500 	NSE: (clipped to 0) -0.40423280294185315
02GC010 	NSE: (clipped to 0) 0.2372790049869713
02GE007 	NSE: (clipped to 0) 0.25160744458980766
02GD004 	NSE: (clipped to 0) 0.27038607749105725
04214500 	NSE: (clipped to 0) -0.17759416510003367
04213000 	NSE: (clipped to 0) 0.38858192684374404
02GG009 	NSE: (clipped to 0) 0.196376052903659
02GA010 	NSE: (clipped to 0) 0.12352870219575784
04176500 	NSE: (clipped to 0) -0.07225779780600328
04177000 	NSE: (clipped to 0) 0.18101217954791438
02GA047 	NSE: (clipped to 0) 0.3602366317550313
02GC018 	NSE: (clipped to 0) 0.04315967660011599
04166500 	NSE: (clipped to 0) 0.35397959344995755
04207200 	NSE: (clipped to 0) 0.4924907861206391
04160600 	NSE: (clipped to 0) 0.16655973951078074
04215000 	NSE: (clipped to 0) 0.2481074848017386
02GA018 	NSE: (clipped to 0) 0.20965547357603753
02GG006 	NS